In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from src.utils import summary_relative_error, statistical_summary

In [3]:
df = summary_relative_error()

In [4]:
pepe = df[df['sampling method'] == 'test_score_dynamic_kdn']['performance gap'].astype(float).describe()

In [5]:
statistical_summary(df, 'performance gap')

,mean,std,min,25%,50%,75%,max
test_score_kdn,0.004348,0.058408,-0.151957,-0.018322,0.0,0.01877,0.224222
test_score_dynamic_kdn,-0.008496,0.049842,-0.371072,-0.01901,-0.001551,0.003704,0.146775
test_score_dynamic_kdn_full,0.004151,0.054837,-0.295845,-0.017968,0.0,0.02912,0.153465


In [6]:
statistical_summary(df, 'sample')

,mean,std,min,25%,50%,75%,max
test_score_kdn,0.53331,0.347244,0.0,0.165,0.565,0.87,0.99
test_score_dynamic_kdn,0.674712,0.299436,0.02,0.5325,0.72,0.94,0.99
test_score_dynamic_kdn_full,0.494628,0.304634,0.0,0.27,0.45,0.72,1.0
